In [3]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import QuantileTransformer, PowerTransformer

from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

import env

from importlib import reload

In [40]:
def acquire_zillow():
    '''
    acuires data from codeup data base
    returns a pandas dataframe with
    'Single Family Residential' properties of 2017
    from zillow
    '''
    
    filename = 'zillow.csv'
    sql = '''
    SELECT bathroomcnt, bedroomcnt, calculatedfinishedsquarefeet, \
        fips, lotsizesquarefeet, poolcnt, \
        yearbuilt, taxvaluedollarcnt
    FROM properties_2017
    JOIN predictions_2017 USING(parcelid)
    JOIN propertylandusetype USING(propertylandusetypeid)
    WHERE propertylandusedesc='Single Family Residential' AND transactiondate LIKE '2017%%'
    '''

    url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/zillow'
    
    # if csv file is available locally, read data from it
    if os.path.isfile(filename):
        df = pd.read_csv(filename) 
    
    # if *.csv file is not available locally, acquire data from SQL database
    # and write it as *.csv for future use
    else:
        # read the SQL query into a dataframe
        df =  pd.read_sql(sql, url)
        
        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename, index_label = False)
        
    return df

In [12]:
df = acquire_zillow()

In [13]:
df

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,lotsizesquarefeet,poolcnt,yearbuilt,taxvaluedollarcnt
0,2.0,3.0,1907.0,6059.0,8592.0,NaN,1959.0,893699.0
1,2.0,3.0,1817.0,6037.0,7485.0,NaN,1985.0,174757.0
2,2.0,3.0,1801.0,6059.0,7200.0,NaN,1965.0,660451.0
3,4.0,4.0,2954.0,6037.0,2396.0,NaN,1924.0,735227.0
4,2.0,4.0,1390.0,6059.0,7000.0,NaN,1962.0,429900.0
...,...,...,...,...,...,...,...,...
52436,3.0,4.0,2133.0,6037.0,5280.0,NaN,1954.0,95220.0
52437,2.5,4.0,2269.0,6059.0,4092.0,NaN,2000.0,485217.0
52438,2.0,3.0,1362.0,6037.0,5601.0,NaN,1958.0,420000.0
52439,2.0,3.0,1282.0,6037.0,7569.0,NaN,1956.0,407768.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   52441 non-null  float64
 1   bedroomcnt                    52441 non-null  float64
 2   calculatedfinishedsquarefeet  52359 non-null  float64
 3   fips                          52441 non-null  float64
 4   lotsizesquarefeet             52072 non-null  float64
 5   poolcnt                       11096 non-null  float64
 6   yearbuilt                     52325 non-null  float64
 7   taxvaluedollarcnt             52440 non-null  float64
dtypes: float64(8)
memory usage: 3.2 MB


In [15]:
df.isnull().sum()

bathroomcnt                         0
bedroomcnt                          0
calculatedfinishedsquarefeet       82
fips                                0
lotsizesquarefeet                 369
poolcnt                         41345
yearbuilt                         116
taxvaluedollarcnt                   1
dtype: int64

In [16]:
def rename_columns(df):
        df.rename(columns={
        'bedroomcnt':'bedrooms',
        'bathroomcnt':'bathrooms',
        'calculatedfinishedsquarefeet':'sq_feet',
        'taxvaluedollarcnt':'home_value',
        'yearbuilt':'year_built',
        'lotsizesquarefeet':'lot_sqft',
        'poolcnt':'pools',
    }, inplace=True)

In [18]:
rename_columns(df)

In [19]:
df.head()

,bathrooms,bedrooms,sq_feet,fips,lot_sqft,pools,year_built,home_value
0,2.0,3.0,1907.0,6059.0,8592.0,NaN,1959.0,893699.0
1,2.0,3.0,1817.0,6037.0,7485.0,NaN,1985.0,174757.0
2,2.0,3.0,1801.0,6059.0,7200.0,NaN,1965.0,660451.0
3,4.0,4.0,2954.0,6037.0,2396.0,NaN,1924.0,735227.0
4,2.0,4.0,1390.0,6059.0,7000.0,NaN,1962.0,429900.0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bathrooms   52441 non-null  float64
 1   bedrooms    52441 non-null  float64
 2   sq_feet     52359 non-null  float64
 3   fips        52441 non-null  float64
 4   lot_sqft    52072 non-null  float64
 5   pools       11096 non-null  float64
 6   year_built  52325 non-null  float64
 7   home_value  52440 non-null  float64
dtypes: float64(8)
memory usage: 3.2 MB


In [22]:
df['pools'] = df.pools.replace({np.NAN:0})

In [23]:
df['lot_sqft'] = df.lot_sqft.replace({np.NAN:0})

In [24]:
df.loc[:, 'bedrooms'].values

array([3., 3., 3., ..., 3., 3., 6.])

In [25]:
df.loc[:, 'bedrooms'] = df.loc[:, 'bedrooms'].values.astype('uint8', copy=False)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bathrooms   52441 non-null  float64
 1   bedrooms    52441 non-null  uint8  
 2   sq_feet     52359 non-null  float64
 3   fips        52441 non-null  float64
 4   lot_sqft    52441 non-null  float64
 5   pools       52441 non-null  float64
 6   year_built  52325 non-null  float64
 7   home_value  52440 non-null  float64
dtypes: float64(7), uint8(1)
memory usage: 2.9 MB


In [27]:
df.isnull().sum()

bathrooms       0
bedrooms        0
sq_feet        82
fips            0
lot_sqft        0
pools           0
year_built    116
home_value      1
dtype: int64

In [28]:
df = df.dropna(axis = 0)

In [29]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52319 entries, 0 to 52440
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bathrooms   52319 non-null  float64
 1   bedrooms    52319 non-null  uint8  
 2   sq_feet     52319 non-null  float64
 3   fips        52319 non-null  float64
 4   lot_sqft    52319 non-null  float64
 5   pools       52319 non-null  float64
 6   year_built  52319 non-null  float64
 7   home_value  52319 non-null  float64
dtypes: float64(7), uint8(1)
memory usage: 3.2 MB


(52319, 8)

In [30]:
df.pools.value_counts()

0.0    41232
1.0    11087
Name: pools, dtype: int64

In [31]:
df.bathrooms.value_counts()

2.0     21888
3.0     10666
1.0      9553
2.5      3931
4.0      2227
3.5       913
1.5       840
5.0       803
4.5       682
6.0       322
5.5       224
7.0        88
8.0        53
6.5        47
0.0        41
7.5        16
9.0        13
10.0        5
11.0        3
8.5         2
18.0        1
13.0        1
Name: bathrooms, dtype: int64

In [33]:
df.loc[:, 'bathrooms'] = df.loc[:, 'bathrooms'].astype('uint8')


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52319 entries, 0 to 52440
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bathrooms   52319 non-null  uint8  
 1   bedrooms    52319 non-null  uint8  
 2   sq_feet     52319 non-null  float64
 3   fips        52319 non-null  float64
 4   lot_sqft    52319 non-null  float64
 5   pools       52319 non-null  float64
 6   year_built  52319 non-null  float64
 7   home_value  52319 non-null  float64
dtypes: float64(6), uint8(2)
memory usage: 2.9 MB


In [35]:
df.loc['lot_sqft'] = df.loc[:, 'lot_sqft'].astype(int)

In [36]:
((df.bathrooms == 0) & (df.bedrooms == 0)).sum()

25

In [37]:
def transform_columns(df):
    # transform fips to integer
    df['fips'] = df.loc[:, 'fips'].astype(int)
    #remove null values   
    #df = df.dropna()

    # add a new column with county names
    df['county_name'] = np.select([(df.fips == 6037), (df.fips == 6059), (df.fips == 6111)],
                             ['LA', 'Orange', 'Ventura'])
    # column to category data type
    df['county_name'] = df.loc[:, 'county_name'].astype('category', copy=False)

    # replace NaN with 0 in the pools columns
    df['pools'] = df.pools.replace({np.NAN:0})
    
    # replace NaN with 0 in the lot_sqft. Needs to be imputed later
    df['lot_sqft'] = df.lot_sqft.replace({np.NAN:0})
    
    # drop null values
    df = df.dropna(axis=0)
    
    # create a column 'house_age'
    df['house_age'] = 2017 - df.year_built

    # change the type of bedrooms/sq_feet/home_value/pools to integer
    df['bedrooms'] = df.loc[:, 'bedrooms'].values.astype('uint8', copy=False)
    df['sq_feet'] = df.loc[:, 'sq_feet'].values.astype(int, copy=False)
    df['lot_sqft'] = df.loc[:, 'lot_sqft'].values.astype(int, copy=False)
    df['home_value'] = df.loc[:, 'home_value'].values.astype(int, copy=False)
    df['house_age'] = df.loc[:, 'house_age'].values.astype('uint8', copy=False)
    df['pools'] = df.loc[:, 'pools'].values.astype('uint8', copy=False)
    

    

    #rearange columns and drop 'fips'
    cols = ['bedrooms',
            'bathrooms',
            'sq_feet',
            'lot_sqft',
            'year_built',
            'house_age',
            'pools',
            'county_name',
            'home_value']
    
    df = df[cols]
    
    return df

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52320 entries, 0 to lot_sqft
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bathrooms   52319 non-null  float64
 1   bedrooms    52319 non-null  float64
 2   sq_feet     52319 non-null  float64
 3   fips        52319 non-null  float64
 4   lot_sqft    52319 non-null  float64
 5   pools       52319 non-null  float64
 6   year_built  52319 non-null  float64
 7   home_value  52319 non-null  float64
dtypes: float64(8)
memory usage: 3.6+ MB


In [39]:
df.head(25)

,bathrooms,bedrooms,sq_feet,fips,lot_sqft,pools,year_built,home_value
0,2.0,3.0,1907.0,6059.0,8592.0,0.0,1959.0,893699.0
1,2.0,3.0,1817.0,6037.0,7485.0,0.0,1985.0,174757.0
2,2.0,3.0,1801.0,6059.0,7200.0,0.0,1965.0,660451.0
3,4.0,4.0,2954.0,6037.0,2396.0,0.0,1924.0,735227.0
4,2.0,4.0,1390.0,6059.0,7000.0,0.0,1962.0,429900.0
5,1.0,2.0,823.0,6037.0,3236.0,0.0,1986.0,148542.0
6,3.0,4.0,2966.0,6037.0,7372.0,1.0,2001.0,654241.0
7,2.0,2.0,1540.0,6037.0,9358.0,0.0,1938.0,456340.0
8,3.0,3.0,1925.0,6037.0,4869.0,0.0,1979.0,522000.0
9,1.0,2.0,959.0,6111.0,7184.0,0.0,1952.0,309818.0
